# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [1]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint
from functions_variables import encode_tags

## Data Importing

In [ ]:
# load one file first to see what type of data you're dealing with and what attributes it has

# Function to flatten nested JSON columns
def flatten_json(record):
    flat_record = record.copy()

    # Flatten 
    if "description" in record and isinstance(record["description"], dict):
        flat_record.update(record["description"])
        del flat_record["description"]
    
    if "location" in record and isinstance(record["location"], dict):
        if "address" in record["location"] and isinstance(record["location"]["address"], dict):
            flat_record.update(record["location"]["address"])
        if "coordinate" in record["location"] and isinstance(record["location"]["coordinate"], dict):
            flat_record["latitude"] = record["location"]["coordinate"].get("lat")
            flat_record["longitude"] = record["location"]["coordinate"].get("lon")
        del flat_record["location"]

    return flat_record

file_path = r"C:\Users\lai29\OneDrive - UBC\Documents\GitHub\Midterm-project\data\AK_Juneau_0.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract and flatten relevant data
if "data" in data and "results" in data["data"]:
    extracted_data = [flatten_json(record) for record in data["data"]["results"]]
else:
    extracted_data = []

# Convert to DataFrame
df_single = pd.DataFrame(extracted_data)

# Display the first few rows of the flattened DataFrame
df_single.head()

    

,primary_photo,last_update_date,source,tags,permalink,status,list_date,open_houses,branding,list_price,...,garage,stories,beds,type,postal_code,state,coordinate,city,state_code,line
0,{'href': 'https://ap.rdcpix.com/07097d34c98a59...,2023-09-19T20:52:50Z,"{'plan_id': None, 'agents': [{'office_name': '...","[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,...,1.0,NaN,3.0,single_family,99801,Alaska,"{'lon': -134.59372, 'lat': 58.36395}",Juneau,AK,9453 Herbert Pl
1,None,None,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,8477 Thunder Mountain Rd
2,None,None,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,4515 Glacier Hwy
3,None,None,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,17850 Point Stephens Rd
4,None,None,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,9951 Stephen Richards Memorial Dr


In [52]:
import os
import json
import pandas as pd

# Directory containing JSON files
directory = r"C:\Users\lai29\OneDrive - UBC\Documents\GitHub\Midterm-project\data"

# Initialize an empty list to store data
data_list = []

# Function to flatten nested JSON columns
def flatten_json(record):
    flat_record = record.copy()

    # Flatten description
    if "description" in record and isinstance(record["description"], dict):
        flat_record.update(record["description"])
        del flat_record["description"]

    # Flatten location
    if "location" in record and isinstance(record["location"], dict):
        if "address" in record and isinstance(record["location"]["address"], dict):
            flat_record.update(record["location"]["address"])
        if "coordinate" in record and isinstance(record["location"]["coordinate"], dict):
            flat_record["latitude"] = record["location"]["coordinate"].get("lat")
            flat_record["longitude"] = record["location"]["coordinate"].get("lon")
        del flat_record["location"]

    return flat_record

# Loop through each JSON file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)

        try:
            with open(file_path, "r", encoding="utf-8") as file:
                data = json.load(file)  # Load JSON data

            # Check if expected keys exist before accessing them
            if isinstance(data, dict) and "data" in data and isinstance(data["data"], dict) and "results" in data["data"]:
                for record in data["data"]["results"]:
                    if isinstance(record, dict):  # Ensure record is a dictionary before processing
                        flat_record = flatten_json(record)
                        data_list.append(flat_record)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")
        except Exception as e:
            print(f"Unexpected error processing file {filename}: {e}")

# Convert list of dictionaries into a DataFrame
df_combined = pd.DataFrame(data_list)

# Display or save the combined DataFrame
print(df_combined.head())  # View first few rows
df_combined.to_csv("combined_data.csv", index=False)

                                       primary_photo      last_update_date  \
0  {'href': 'https://ap.rdcpix.com/07097d34c98a59...  2023-09-19T20:52:50Z   
1                                               None                  None   
2                                               None                  None   
3                                               None                  None   
4                                               None                  None   

                                              source  \
0  {'plan_id': None, 'agents': [{'office_name': '...   
1                                               None   
2                                               None   
3                                               None   
4                                               None   

                                                tags  \
0  [carport, community_outdoor_space, cul_de_sac,...   
1                                               None   
2                         

In [53]:
df_combined.head()

,primary_photo,last_update_date,source,tags,permalink,status,list_date,open_houses,branding,list_price,...,baths_half,lot_sqft,sqft,baths,sub_type,baths_1qtr,garage,stories,beds,type
0,{'href': 'https://ap.rdcpix.com/07097d34c98a59...,2023-09-19T20:52:50Z,"{'plan_id': None, 'agents': [{'office_name': '...","[carport, community_outdoor_space, cul_de_sac,...",9453-Herbert-Pl_Juneau_AK_99801_M90744-30767,sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,...,NaN,10454.0,1821.0,2.0,None,None,1.0,NaN,3.0,single_family
1,None,None,None,None,8477-Thunder-Mountain-Rd_Juneau_AK_99801_M9424...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None
2,None,None,None,None,4515-Glacier-Hwy_Juneau_AK_99801_M94790-68516,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None
3,None,None,None,None,17850-Point-Stephens-Rd_Juneau_AK_99801_M98793...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None
4,None,None,None,None,9951-Stephen-Richards-Memorial-Dr_Juneau_AK_99...,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None


In [50]:
df_combined.describe()

,list_price,price_reduced_amount,year_built,baths_3qtr,sold_price,baths_full,baths_half,lot_sqft,sqft,baths,garage,stories,beds
count,7.721000e+03,2.484000e+03,7316.000000,566.000000,6.716000e+03,7311.000000,2281.000000,6.991000e+03,7323.000000,7980.000000,4448.000000,6260.000000,7504.000000
mean,4.341582e+05,2.442704e+04,1968.916074,1.247350,4.126050e+05,1.887430,1.105655,2.510949e+05,1933.848559,2.131203,1.926709,1.567732,3.208289
std,5.514925e+05,7.162396e+04,35.096914,0.463482,6.994308e+05,0.862214,0.413340,5.823820e+06,1339.039206,1.175940,0.878766,0.730969,1.282732
min,1.000000e+00,1.000000e+02,1828.000000,1.000000,3.080000e+02,1.000000,1.000000,0.000000e+00,120.000000,0.000000,1.000000,1.000000,0.000000
25%,2.090000e+05,6.000000e+03,1950.000000,1.000000,1.910000e+05,1.000000,1.000000,4.953000e+03,1258.000000,1.000000,1.000000,1.000000,3.000000
50%,3.250000e+05,1.010000e+04,1975.000000,1.000000,3.140000e+05,2.000000,1.000000,7.841000e+03,1635.000000,2.000000,2.000000,1.000000,3.000000
75%,4.999000e+05,2.000000e+04,1997.000000,1.000000,4.700000e+05,2.000000,1.000000,1.263200e+04,2264.000000,3.000000,2.000000,2.000000,4.000000
max,1.250000e+07,2.015999e+06,2024.000000,3.000000,2.706500e+07,8.000000,5.000000,1.676624e+08,32106.000000,9.000000,11.000000,10.000000,12.000000


In [56]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8159 entries, 0 to 8158
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   primary_photo         7403 non-null   object 
 1   last_update_date      8125 non-null   object 
 2   source                7752 non-null   object 
 3   tags                  7638 non-null   object 
 4   permalink             8159 non-null   object 
 5   status                8159 non-null   object 
 6   list_date             7752 non-null   object 
 7   open_houses           0 non-null      object 
 8   branding              8159 non-null   object 
 9   list_price            7721 non-null   float64
 10  lead_attributes       8159 non-null   object 
 11  property_id           8159 non-null   object 
 12  photos                7403 non-null   object 
 13  flags                 8159 non-null   object 
 14  community             5 non-null      object 
 15  products             

In [55]:
df_combined.dtypes

#Create a list of categorical variables
categorical = df_combined.select_dtypes(include=['object']).columns

#Create a list of numerical variables
numerical = df_combined.select_dtypes(exclude=['object']).columns

print(categorical)
print(numerical)

Index(['primary_photo', 'last_update_date', 'source', 'tags', 'permalink',
       'status', 'list_date', 'open_houses', 'branding', 'lead_attributes',
       'property_id', 'photos', 'flags', 'community', 'products',
       'virtual_tours', 'other_listings', 'listing_id', 'sold_date', 'name',
       'sub_type', 'baths_1qtr', 'type'],
      dtype='object')
Index(['list_price', 'price_reduced_amount', 'matterport', 'year_built',
       'baths_3qtr', 'sold_price', 'baths_full', 'baths_half', 'lot_sqft',
       'sqft', 'baths', 'garage', 'stories', 'beds'],
      dtype='object')


In [ ]:
#Remove initial columns 

df_dropped = df_combined.drop(columns=['primary_photo', 'last_update_date','source','permalink','branding', ])

KeyError: "['primary_photo', 'last_update_date', 'source', 'permalink'] not found in axis"

In [47]:
df_combined.head()
#Columns to drop: tags, list date?, branding, lead attributes, propertyID, photos, flags, listing_id

,tags,status,list_date,open_houses,branding,list_price,lead_attributes,property_id,photos,flags,...,garage,stories,beds,type,postal_code,state,coordinate,city,state_code,line
0,"[carport, community_outdoor_space, cul_de_sac,...",sold,2023-06-29T21:16:25.000000Z,None,"[{'name': 'EXP Realty LLC - Southeast Alaska',...",554950.0,{'show_contact_an_agent': True},9074430767,"[{'tags': [{'label': 'house_view', 'probabilit...","{'is_new_construction': None, 'is_for_rent': N...",...,1.0,NaN,3.0,single_family,99801,Alaska,"{'lon': -134.59372, 'lat': 58.36395}",Juneau,AK,9453 Herbert Pl
1,None,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,{'show_contact_an_agent': True},9424983842,None,"{'is_new_construction': None, 'is_for_rent': N...",...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,8477 Thunder Mountain Rd
2,None,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,{'show_contact_an_agent': True},9479068516,None,"{'is_new_construction': None, 'is_for_rent': N...",...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,4515 Glacier Hwy
3,None,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,{'show_contact_an_agent': True},9879331943,None,"{'is_new_construction': None, 'is_for_rent': N...",...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,17850 Point Stephens Rd
4,None,sold,None,None,"[{'name': None, 'photo': None, 'type': 'Office'}]",NaN,{'show_contact_an_agent': True},9521639574,None,"{'is_new_construction': None, 'is_for_rent': N...",...,NaN,NaN,NaN,None,99801,Alaska,None,Juneau,AK,9951 Stephen Richards Memorial Dr


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [ ]:
# load and concatenate data here
# drop or replace values as necessary

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [ ]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [ ]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [ ]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [ ]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized